# Phân tích đa biến
Trong tuần này chúng ta sẽ tiến hành phân tích mối tương quan giữa các biến trong một bảng dữ liệu để thấy được mối quan hệ. Từ đó, cho phép chọn lựa các đặc trưng phù hợp để giải quyết bài toán.

Đầu tiên, tương tự như các bài tập trước, ta sẽ import những thư viện cần thiết

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Ở bài này, ta sẽ EDA những feature để phục vụ cho cho mô hình dự đoán (predict), trong đó:
* Output (giá trị cần dự đoán) là cột `Survived`.
* Các feature còn lại (ví dụ: `Pclass`, `Name`, `Sex`, `Age`) đóng vai trò là input của mô hình.

*Việc đầu tiên cần làm* là đọc file dữ liệu csv bằng thư viện `pandas` như những bài trước và lưu dataframe vào biến `df`

In [ ]:
df = pd.read_csv("./data/train.csv")
df.head()

## 1. Thống kê phân bố theo giá trị output

Tiếp theo, ta phân tích output feature có tên là `Survived` để quan sát tập giá trị đầu ra và phân bố của các giá trị này:

In [ ]:
sns.countplot(x=df['Survived']);

Dựa vào biểu đồ ta có thể thấy, đây là bài toán phân loại nhị phân (binary classification) với hai nhãn đầu ra cần dự đoán là `0` và `1`. Trong đó, số lượng nhãn 0 chiếm tỉ trọng cao hơn so với nhãn 1.

Tiếp đến, ta sẽ phân tích xem **những cột còn lại có liên quan gì đến cột `Survived` này hay không**.

Đầu tiên là thống kê phân bố của nhãn cần dự đoán theo cột `Sex` (giới tính):

In [ ]:
sns.countplot(x=df['Sex'], hue=df['Survived']);

**Nhận xét**: Dựa vào biểu đồ trên, ta thấy tỉ lệ nữ còn sống so với nữ đã mất cao hơn tỉ lệ đó trên tập nam giới.

Ta thấy việc phân tích từng cột thế này **quá mất thời gian**, ta quyết định phân tích và hiển thị cùng lúc tất cả các cột có dạng dữ liệu Category hoặc những cột dạng Numerical mà ít labels như: [`Pclass`, `Sex`, `SibSp`, `Parch`, `Embarked`] bằng hàm `countplot` của thư viện `seaborn` và `subplot` của thư viện `matplotlib`.

Với từng subplot (của thư viện `matplotlib`), ta dùng hàm `sns.countplot(x=..., hue=..., ax=...)` để vẽ các biểu đồ phân tích. Trong đó cần phải truyền vào:
* Tham số `x` là những cột theo thứ tự trong cols, và chạy từ trên xuống dưới, trái qua phải.
* Tham số `hue` = cột 'Survived'

Các bạn cần hoàn thành bằng cách duyệt từ trên xuống dưới, từ trái qua phải các subplot để vẽ từng biểu đồ phân tích phân bố của giá trị output theo từng biến input đã nêu ở trên.

Vì số lượng ô của subplot là 2 dòng x 3 cột = 6 ô, trong khi số lượng đặc trưng cần hiển thị chỉ có 5, do đó ta cần phải xoá ô cuối cùng. Để xóa ô cuối cùng này ta sử dụng hàm `set_visible`. Đoạn code này đã được cung cấp ngay phía sau.

In [ ]:
# Danh sách  5 cột cần hiển thị theo thứ tự
cols = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']

# Tạo subplot kích thước 2 x 3
fig, ax = plt.subplots(2, 3, figsize=(18, 18))
index = 0

### BEGIN SOLUTION
for i in range(2):
    for j in range(3):
        if index < len(cols):
            sns.countplot(x=df[cols[index]], hue=df['Survived'], ax=ax[i, j])
            index += 1
### END SOLUTION

# Xóa ô cuối cùng vì chỉ có 5 cột cần hiển thị
ax.flat[-1].set_visible(False)
plt.tight_layout()

fig.savefig('part_1.png')

**Nhận xét**:
  * Ở cột `PClass`:
    - Class 3 có tỉ lệ chết (0) cao hơn đột biến so với tỉ lệ sống (1)
    - Class 2 có tỉ lệ chết và sống gần như bằng nhau
    - Class 1 có tỉ lệ sống cao hơn
  * Ở cột `SibSp`:
    - Đa số tỉ lệ chết (0) đều cao hơn tỉ lệ sống (1)
    - Riêng ở cột có giá trị 1 thì tỉ lệ sống cao hơn đôi chút
  * Ở cột `Parch`:
    - Đa số tỉ lệ chết (0) đều cao hơn tỉ lệ sống (1)
    - Riêng ở cột có giá trị 1 thì tỉ lệ sống cao hơn đôi chút
  * Ở cột `Embarked`:
    - Vẫn giống những cột ở trên, ở cột này tỉ lệ chết (0) vẫn trội hơn so với sống (1)
  **Ta nhận thấy cột `SibSp` và `Parch` gần như giống nhau về sự ảnh hưởng đối với cột `Survived`*

Tiếp theo là những cột ['Age', 'Fare'], đây là dữ liệu dạng numerical (số), nếu ta dùng countplot như trên sẽ rất khó quan sát và phân tích:

In [ ]:
sns.countplot(x=df['Age'])

Do đó, ở đây mình sẽ dùng hàm `histplot` của thư viện `seaborn` để phân tích. Trong đó, cần phải truyền vào:
* Tham số `x` là cột `Age`
* Tham số `hue` là cột `Survived`
* Tham số `kde`=`True`: tham số để phân tích dạng đường phân bố liên tục.

Kết quả của việc phân tích này được lưu vào biến `check1_sv` = `sns.histplot(...)`.






In [ ]:
plt.figure(figsize=(12, 12))
### BEGIN SOLUTION
check1_sv = sns.histplot(x = df['Age'], hue = df['Survived'], kde = True)
### END SOLUTION

plt.savefig('part_2.png')

**Nhận xét**: Ta có thể nhận xét, từ khoảng 0 đến 10 tuổi thì tỉ lệ sống sót cao hơn tỉ lệ mất.

Tương tự, ta sẽ phân tích cột `Fare` so với cột output `Survived`:

In [ ]:
sns.histplot(x=df['Fare'], hue=df['Survived'], kde=True);

**Nhận xét**: Ta thấy nó khá khó nhìn nên ta sẽ dùng phương pháp binning Quantile.

Do đó, ta sẽ dùng hàm `countplot` của `Seaborn` để phân tích. Trong đó, cần phải truyền vào:
* Tham số `x`=`bins`, với `bins` là kết quả của quá trình binning (đã được cung cấp)
* Tham số `hue`= cột `Survived`

Kết quả của phân tích này ta lưu vào biến `check2_sv`.

In [ ]:
labels = ['Cheap', 'Expensive', 'Luxury']
bins = pd.qcut(df['Fare'], 3, labels=labels)
sns.countplot(x=bins)

plt.figure(figsize=(12,12))

### BEGIN SOLUTION
check2_sv = sns.countplot(x=bins, hue=df['Survived'])
### END SOLUTION

plt.savefig('part_3.png')

**Nhận xét:** Ta nhận thấy ở mức giá vé cao hơn thì tỉ lệ sống sót sẽ được tăng mạnh

## 2. Tính toán Correlation

Ta xem lại dataframe và xem dataframe bị khuyết bao nhiêu dữ liệu

In [ ]:
df

In [14]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

* Ta sẽ phải xoá đi cột `PassengerId` và cột `Name` vì 2 cột này sẽ không thể ảnh hưởng đến việc sống hoặc chết
* Ở khuôn khổ bài này, ta tạm thời không nói đến việc điền khuyết dữ liệu trống, nên những dòng bị khuyết dữ liệu, ta xoá đi
* Ta dùng kĩ thuật Ordinal Encoding ở cột `Sex` và `Embarked`

In [15]:
### BEGIN SOLUTION
from sklearn.preprocessing import OrdinalEncoder

# Xóa cột 'PassengerId' và 'Name'
df.drop(['PassengerId', 'Name'], axis=1, inplace=True)

# Xóa các dòng có dữ liệu trống
df.dropna(inplace=True)

# Áp dụng Ordinal Encoder cho cột Sex và Embarked
encoder = OrdinalEncoder()
df[['Sex', 'Embarked']] = encoder.fit_transform(df[['Sex', 'Embarked']])

### END SOLUTION

Mục đích của việc làm trên là để biến dữ liệu dạng category thành numerical và bỏ hết giá trị NaN để tính toán correlation giữa các feature

In [ ]:
heatmap_val = df.corr()
heatmap_val

Ta dùng hàm `heatmap` của thư viện `seaborn` để vẽ heatmap.

In [ ]:
plt.figure(figsize=(12,12))
### BEGIN SOLUTION

features = ["Age", "Embarked", "Fare", "Parch", "Pclass", "Sex", "SibSp", "Survived"]
heatmap_val = df[features].corr()
sns.heatmap(heatmap_val, annot=True)

### END SOLUTION

plt.savefig('part_4.png')

Không ngoài dự đoán, 2 feature `SibSp` và `Parch` có độ tương đồng cao nhất (0.38) trong các feature